# A variational autoencoder

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Lambda, Input, Dense
#from tensorflow.keras.models import Model
from keras import Model

from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import tensorflow as tf

from keras.models import Sequential
from keras.layers import *
from keras import layers, models

def to_functional_model(seqmodel):
    input_layer = Input(batch_shape=seqmodel.layers[0].input_shape)
    prev_layer = input_layer
    for layer in seqmodel.layers:
        prev_layer = layer(prev_layer)
    output_layer = prev_layer
    funcmodel = models.Model([input_layer], [prev_layer])
    return input_layer, output_layer, funcmodel

def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def get_vae_loss(inputs, outputs, z_mean, z_log_var, reconstruction_loss = "mse", original_dim = 130, impact_reconstruction_loss = 1):
    # VAE loss = mse_loss or binary_crossentropy + kl_loss
    if reconstruction_loss == "binary_crossentropy":
        reconstruction_loss = binary_crossentropy(inputs,outputs)
    elif reconstruction_loss == "mse":
        reconstruction_loss = mse(inputs, outputs)

    reconstruction_loss *= original_dim
    
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(K.mean(impact_reconstruction_loss*reconstruction_loss) + K.mean(kl_loss))
    return vae_loss

def image_encoder():
    image_model = Sequential()
    image_model.add(Conv2D(32, (8, 8), subsample=(4, 4), input_shape=[50, 50, 1], activation = "relu"))
    image_model.add(Conv2D(64, (4, 4), subsample=(2, 2), activation = "relu"))
    image_model.add(Conv2D(64, (3, 3), subsample=(1, 1), activation = "relu"))
    image_model.add(Flatten())
    image_model.add(Dense(512, activation = "relu"))
    image_model.add(Dense(512, activation = "relu"))
    image_input, image_output, image_model = to_functional_model(image_model)
    return image_input, image_output

def encoder():
    image_input, image_output = image_encoder()
    z_mean = Dense(2, name='z_mean')(image_output)
    z_log_var = Dense(2, name='z_log_var')(image_output)
    z = Lambda(sampling, output_shape=(2,), name='z')([z_mean, z_log_var])
    return image_input, [z_mean, z_log_var, z]

def image_decoder(encoded):
    shape = [-1,16,16, 1]
    x = Dense(256, activation = "relu")(encoded)
    x = Dense(256, activation = "relu")(encoded)
    x = Reshape((shape[1], shape[2], shape[3]))(x)
    x = layers.Conv2D(8, (6, 6), activation='relu', padding = "same")(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(5, (6, 6), activation='relu')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(1, (5, 5), activation='relu')(x)
    return x


def decoder():
    latent_inputs = Input(shape=(2,), name='z_sampling')
    image_decoded = image_decoder(latent_inputs)
    return latent_inputs, image_decoded

Using TensorFlow backend.


In [2]:
image_input, [z_mean, z_log_var, z] = encoder()
latent_inputs, image_decoded = decoder()
encoder_model = Model(inputs = image_input, outputs = [z_mean, z_log_var, z], name = "encoder")
decoder_model = Model(inputs = latent_inputs, outputs = image_decoded, name='decoder')

/home/elerator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=[50, 50, 1..., activation="relu", strides=(4, 4))`
/home/elerator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), activation="relu", strides=(2, 2))`
/home/elerator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1))`


In [3]:
image_decoded.shape

TensorShape([None, 50, 50, 1])

In [4]:
image_decoded = decoder_model(encoder_model(image_input)[2])

In [5]:
vae = Model(inputs = image_input, outputs = image_decoded, name='vae_cnn')

In [6]:
import keras
vae.add_loss(get_vae_loss(image_input, image_decoded, z_mean, z_log_var, impact_reconstruction_loss = 1000))
vae.compile(optimizer = "adam")

/home/elerator/anaconda3/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


# Load dataset

In [7]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

dataset_folder = "texture_datasets"
source_patches = np.load(os.path.join(dataset_folder,"source_patches.npy"))
labels = np.load(os.path.join(dataset_folder,"labels.npy"))

In [8]:
x_train, x_test, y_train, y_test = train_test_split(source_patches, labels, test_size = 0.2, random_state=42)

In [9]:
x_train = np.expand_dims(x_train, -1)
y_train = np.expand_dims(y_train, -1)

In [10]:
x_train.shape

(35528, 50, 50, 1)

In [11]:
batch_size = 100
epochs = 400

In [12]:
history = vae.fit(x_train[:1000], epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/400
1000/1000 [==============================] - 14s 14ms/step - loss: 26962.7907
Epoch 2/400
1000/1000 [==============================] - 4s 4ms/step - loss: 11916.2411
Epoch 3/400
1000/1000 [==============================] - 6s 6ms/step - loss: 7306.8221
Epoch 4/400
1000/1000 [==============================] - 7s 7ms/step - loss: 5609.1581
Epoch 5/400
1000/1000 [==============================] - 8s 8ms/step - loss: 4782.2308
Epoch 6/400
1000/1000 [==============================] - 8s 8ms/step - loss: 4295.3643
Epoch 7/400
1000/1000 [==============================] - 5s 5ms/step - loss: 4027.0140
Epoch 8/400
1000/1000 [==============================] - 4s 4ms/step - loss: 3832.7305
Epoch 9/400
1000/1000 [==============================] - 9s 9ms/step - loss: 3757.3629
Epoch 10/400
1000/1000 [==============================] - 7s 7ms/step - loss: 3719.2612
Epoch 11/400
1000/1000 [==============================] - 6s 6ms/step - loss: 3694.2454
Epoch 12/400
1000/1000 [=============

1000/1000 [==============================] - 2s 2ms/step - loss: 3453.3973
Epoch 95/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3450.6527
Epoch 96/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3451.4121
Epoch 97/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3450.6168
Epoch 98/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3452.4144
Epoch 99/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3462.4527
Epoch 100/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3449.3492
Epoch 101/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3450.3516
Epoch 102/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3461.1199
Epoch 103/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3457.3969
Epoch 104/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3456.6032
Epoch 105/400
1000/1000 [===============

1000/1000 [==============================] - 2s 2ms/step - loss: 3404.6671
Epoch 187/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3394.0082
Epoch 188/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3388.3476
Epoch 189/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3389.3336
Epoch 190/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3388.5758
Epoch 191/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3392.7208
Epoch 192/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3396.8133
Epoch 193/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3386.9728
Epoch 194/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3393.8568
Epoch 195/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3395.1015
Epoch 196/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3387.7855
Epoch 197/400
1000/1000 [==========

1000/1000 [==============================] - 3s 3ms/step - loss: 3376.8123
Epoch 279/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3377.1042
Epoch 280/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3380.1238
Epoch 281/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3381.2144
Epoch 282/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3381.6541
Epoch 283/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3380.3525
Epoch 284/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3378.5116
Epoch 285/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3381.6022
Epoch 286/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3379.7429
Epoch 287/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3384.0309
Epoch 288/400
1000/1000 [==============================] - 4s 4ms/step - loss: 3387.5127
Epoch 289/400
1000/1000 [==========

1000/1000 [==============================] - 2s 2ms/step - loss: 3367.5216
Epoch 371/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3372.2045
Epoch 372/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3370.5406
Epoch 373/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3378.6459
Epoch 374/400
1000/1000 [==============================] - 3s 3ms/step - loss: 3368.4869
Epoch 375/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3370.3328
Epoch 376/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3368.3602
Epoch 377/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3373.1207
Epoch 378/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3377.6709
Epoch 379/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3395.7352
Epoch 380/400
1000/1000 [==============================] - 2s 2ms/step - loss: 3392.1190
Epoch 381/400
1000/1000 [==========

In [13]:
plt.plot(vae.history.history["loss"])

NameError: name 'plt' is not defined

In [ ]:
history = vae.fit(x_train[1000:2000], epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
history = vae.fit(x_train, epochs=1, batch_size=batch_size, verbose=1)

In [ ]:
plt.plot(vae.history.history["loss"])

In [ ]:
# train the autoencoder
#history = vae.fit(x_train, epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
res = encoder_model.predict(x_train)[2]

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
fig, ax = plt.subplots(1)
xy = np.vstack([res.T[0], res.T[1]])
z = gaussian_kde(xy)(xy)

print(len(z))

ax.scatter(res.T[0], res.T[1], c = z, edgecolor = "")
xrange = [np.min(res.T[0]), np.max(res.T[0])]
yrange = [np.min(res.T[1]), np.max(res.T[1])]
ax.set_xlim(xrange)
ax.set_ylim(yrange)

In [ ]:
len(y_train)

In [ ]:
fig, ax = plt.subplots(1)

ax.scatter(res.T[0][~y_train[:,0]], res.T[1][~y_train[:,0]], c = "yellow", edgecolor = "", s = 5, label = "full")
ax.scatter(res.T[0][y_train[:,0]], res.T[1][y_train[:,0]], c = "blue", edgecolor = "", s = 1, label = "empty")
ax.legend()

xrange = [np.min(res.T[0]), np.max(res.T[0])]
yrange = [np.min(res.T[1]), np.max(res.T[1])]
ax.set_xlim(xrange)
ax.set_ylim(yrange)

In [ ]:
n = 25
xs = np.linspace(xrange[0],xrange[1],n)
ys = np.linspace(yrange[0],yrange[1],n)

In [ ]:
test = np.array([[0,0,0],[0,0,0]]).T

In [ ]:
fig, ax = plt.subplots(n,n, figsize= (n,n))
for i in range(len(list(ax))):
    for i1 in range(len(list(ax)[0])):
        ax[i, i1].axis("off")
        ax[i, i1].imshow(decoder_model.predict(np.array([[xs[i]], 
                                                         [ys[i]]]).T)[0,:,:,0], vmin = .25, vmax = .75)